In [2]:
import pandas as pd
import numpy as np
import os
import re
import multiprocessing
import pickle
import tqdm
from itertools import chain

# 학습 데이터 만들기

사전에 뉴스 타이틀과 내용을 문장('.')단위로 분리하여 하나의 텍스트 파일로 만들어놓는다

## 모델의 인풋은 문서 단위가 좋을까 문장 단위가 좋을까?

https://datascience.stackexchange.com/questions/8753/what-is-a-better-input-for-word2vec

-> output으로 뭘 원하는 지에 따라 다름.. 그때그때 다르다.

여기서는 문장 단위로 나눠서 넣어보자

In [10]:
# load pickle
news_corpus = pd.read_pickle('embedding/data/52,320 news title and content _split_by_sent.txt')

In [11]:
news_corpus[:5]

["[美친시청률] '뉴스룸', 또 최고시청률 8.8%..이쯤되면 신드롬\n",
 '[OSEN=표재민 기자] JTBC 간판 뉴스프로그램 ‘뉴스룸’이 또 다시 자체최고시청률을 경신했다',
 '이쯤 되면 손석희와 ‘뉴스룸’ 신드롬이라고 해도 무리가 없을 듯 하다',
 '1일 시청률조사회사 닐슨코리아에 따르면 지난 달 31일 방송된 ‘뉴스룸’은 전국 기준 8.784%를 기록, 종합편성채널이라 기준은 다르지만 동시간대 방송된 SBS ‘8뉴스’(5.9%), MBC ‘뉴스데스크’(5.3%) 등을 제쳤다',
 '종합편성채널 집계 기준과 지상파 시청률 집계 기준이 달라 단순한 비교는 어렵지만 사실상 ‘뉴스룸’을 보는 시청자들이 압도적으로 많다는 것을 간접적으로 알 수 있는 성적표다']

In [7]:
print(len(news_corpus))

524202


In [18]:
from konlpy.tag import Mecab
mecab = Mecab()

def extract_keywords(mecab, sentence):
    """
    품사 분석을 진행한 뒤 관계언(조사 등)이나 기호를 제거한다.
    """
    tagged = mecab.pos(sentence)

    result = []
    
    for word, tag in tagged:
        if tag in ['NNG','NNP','VV','VA']: # 일반명사, 고유명사, 동사, 형용사 사용
            result.append(word)
    return result

print(news_corpus[1])
print(extract_keywords(mecab, news_corpus[1]))

[OSEN=표재민 기자] JTBC 간판 뉴스프로그램 ‘뉴스룸’이 또 다시 자체최고시청률을 경신했다
['표', '재민', '기자', '간판', '뉴스', '프로그램', '뉴스룸', '자체', '최고', '시청', '경신']


In [90]:
news_corpus_token= list(map(lambda x : extract_keywords(mecab, x), news_corpus))

In [92]:
print(news_corpus_token[:5])

[['美', '시청', '뉴스룸', '최고', '시청', '되', '신드롬'], ['표', '재민', '기자', '간판', '뉴스', '프로그램', '뉴스룸', '자체', '최고', '시청', '경신'], ['되', '손석희', '뉴스룸', '신드롬', '무리', '없', '하'], ['시청', '조사', '회사', '닐슨', '코리아', '따르', '달', '방송', '뉴스룸', '전국', '기준', '기록', '종합', '편성', '채널', '기준', '다르', '동시', '방송', '뉴스', '뉴스데스크'], ['종합', '편성', '채널', '집계', '기준', '지상파', '시청', '집계', '기준', '단순', '비교', '어렵', '사실', '뉴스룸', '보', '시청자', '압도', '많', '간접', '있', '성적표']]


In [93]:
# # save as pickle

# with open('../Data/embedding/data/52,320 news title and content _tokenized.txt', 'wb') as f:
#     pickle.dump(news_corpus_token, f)

# Word Embedding 
- word2vec
- fastText

**Reference**    
- [Finding best fasttext hyperparameters](http://soner.in/fasttext-grid-search/)
- [한국어 단어 임베딩을 위한 Word2vec 모델의 최적화](http://journal.dcs.or.kr/xml/19540/19540.pdf)
- [FastText, Word representation using subword](https://lovit.github.io/nlp/representation/2018/10/22/fasttext_subword/)

## word2vec

In [12]:
import multiprocessing
import os

import gensim
from gensim.models import word2vec
from gensim.models import fasttext

In [13]:
config = {
    'window' : 5,
    'min_count': 5,  # 등장 횟수가 5 이하인 단어는 무시
    'size': 150,  # 150차원짜리 벡터스페이스에 embedding
    'sg': 1,  # 0이면 CBOW, 1이면 skip-gram을 사용한다
    'batch_words': 10000,  # 사전을 구축할때 한번에 읽을 단어 수
    'iter': 10,  # 보통 딥러닝에서 말하는 epoch과 비슷한, 반복 횟수
    'workers': multiprocessing.cpu_count(),
}

In [99]:
# 모델 학습
w2v_model = word2vec.Word2Vec(news_corpus_token,**config) # Word2vec 모델 생성

In [ ]:
# 모델 저장
w2v_model.save('../Data/embedding/model/word2vec_model') # 모델을 'model' 파일에 저장

In [ ]:
# 모델 불러오기
w2v_model = word2vec.Word2Vec.load(os.path.join('embedding',"w2v_0327.model"))

In [16]:
w2v_model.wv.most_similar(positive = "중국")

[('짱깨', 0.7469527721405029),
 ('짱개', 0.7402806878089905),
 ('중공', 0.7356770038604736),
 ('짱꼴라', 0.6442628502845764),
 ('러시아', 0.6201988458633423),
 ('중국인', 0.6164897084236145),
 ('중궈', 0.5840282440185547),
 ('시진핑', 0.5833264589309692),
 ('일본', 0.560553789138794),
 ('미국', 0.5600796937942505)]

In [17]:
w2v_model.wv.most_similar(positive = "일본")

[('쪽바리', 0.637258768081665),
 ('일본도', 0.6027649641036987),
 ('롯게', 0.5917129516601562),
 ('좃선님', 0.5848301649093628),
 ('일본인', 0.5770894289016724),
 ('쪽발이', 0.5697472095489502),
 ('중국', 0.560553789138794),
 ('왜놈', 0.5594272613525391),
 ('넘듬', 0.5572184324264526),
 ('짓맓고', 0.5498552918434143)]

In [18]:
w2v_model.wv.most_similar(positive = "대통령")

[('대통', 0.7586009502410889),
 ('문재인', 0.6966331005096436),
 ('통령', 0.6032859086990356),
 ('영부인', 0.6013497710227966),
 ('총리', 0.5542508363723755),
 ('망녕났군', 0.5260519981384277),
 ('박성욱', 0.5248264670372009),
 ('김정숙', 0.5233049392700195),
 ('땔래야땔수없', 0.5201804041862488),
 ('부럽쥬', 0.5174517631530762)]

# fastText

## Gensim 사용

In [15]:
ft_config = {
    'min_count': 1,  # 등장 횟수가 1 이하인 단어는 무시
    'size': 300,  # 300차원짜리 벡터스페이스에 embedding
    'sg': 1,  # 0이면 CBOW, 1이면 skip-gram을 사용한다
    'batch_words': 10000,  # 사전을 구축할때 한번에 읽을 단어 수
    'iter': 10,  # 보통 딥러닝에서 말하는 epoch과 비슷한, 반복 횟수
    'workers': multiprocessing.cpu_count(),
    'window': 5,
    'seed': 25, #random number,
    'word_ngrams':1 # uses enriches word vectors with subword(n-grams) information. If 0, this is equivalent to Word2Vec.
}

In [15]:
# 모델 학습
fastText_model = fasttext.FastText(sentences=tokens, **ft_config)  

In [20]:
# 연관성 테스트
fastText_model.wv.most_similar("중국")

[('짱개', 0.7340356707572937),
 ('짱깨', 0.7271621227264404),
 ('중공', 0.7174848318099976),
 ('러시아', 0.6323301792144775),
 ('짱꼴라', 0.6283606290817261),
 ('중국인', 0.6042678356170654),
 ('중궈', 0.5787336826324463),
 ('시진핑', 0.575260579586029),
 ('일본', 0.5739111304283142),
 ('중국스립', 0.5734913349151611)]

In [16]:
# 모델 저장
fastText_model.save('dataset/embedding/fastText_0330.model')

In [48]:
# 모델 불러오기
fastText_model = fasttext.FastText.load('dataset/embedding/fastText_0330.model')

## fastText API
https://github.com/facebookresearch/fastText/tree/master/python

In [1]:
import fasttext

In [ ]:
# Skipgram model :
fastText_model = fasttext.train_unsupervised(
    input ='embedding/data/52,320 news title and content _split_by_sent.txt', 
    model='skipgram',
    minCount = 5, 
    ws=3, 
    dim = 100)

fastText_model.get_input_matrix().shape

In [113]:
fastText_model.get_nearest_neighbors("문재인")

[(0.880488932132721, '?문재인'),
 (0.8631366491317749, '(문재인'),
 (0.8565274477005005, '‘문재인'),
 (0.8242815732955933, '"(문재인'),
 (0.8241071105003357, '“문재인'),
 (0.8146178126335144, '문재인이'),
 (0.799881637096405, '문재인에'),
 (0.7855387330055237, "문재인'"),
 (0.7854195237159729, "'문재인"),
 (0.7715805768966675, '문재인을')]

In [114]:
fastText_model.save_model("../Data/embedding/model/fastText_model.bin")

In [ ]:
fastText_model = fasttext.load_model("../Data/embedding/model/fastText_model.bin")